In [6]:
df_QUERY_notasExport = pd.read_excel(
    'uploads//nota_export.xlsx')

In [1]:
from flask import Flask, render_template, send_from_directory, request, redirect, url_for, Response
import os
from zipfile import ZipFile
import shutil
import subprocess
from werkzeug.utils import secure_filename
import pandas as pd
import numpy as np
import time
from datetime import datetime


In [9]:
print("leituradf_QUERY_z369")
df_QUERY_z369 = pd.read_json(
    'files\df_stage_Nota.json', lines=True)
print("leituradf_notasExport")
df_QUERY_notasExport = pd.read_excel(
    'uploads//nota_export.xlsx')
print("leituradf_QUERY_TELA_164_FOTO")
df_QUERY_TELA_164_FOTO = pd.read_json(
    'files\df_QUERY_TELA_164_FOTO.json', lines=True)


# df_QUERY_TELA_164_FOTO_teste = df_QUERY_TELA_164_FOTO[(df_QUERY_TELA_164_FOTO['TREM'] == 'C65')]
# print("C65")
# print(len(df_QUERY_TELA_164_FOTO_teste))

if len(df_QUERY_notasExport) > 1:
    print("nota_export OK ")
    # Obter os nomes atuais das colunas
    colunas_atuais = df_QUERY_notasExport.columns
    df_QUERY_notasExport_filtred = df_QUERY_notasExport[(df_QUERY_notasExport['Status do sistema'] == 'MSPN') | (
        df_QUERY_notasExport['Status do sistema'] == 'MSPR')]

    # Mapeamento entre nomes antigos e novos
    mapa_colunas = {
        'Data da nota': 'DT_NOTA',
        'Hora da nota': 'Hora da nota',
        'Equipam.': 'ATIVO_TL',
        'Parada': 'Flag',
        'Notificador': 'Notificador',
        'TpNota': 'TP_NOTA',
        'CódFalha': 'CodFalha',
        'Descrição': 'TEXTO',
        'Texto do item': 'Texto_Item',
        'Texto da causa': 'Texto_Causa',
        'Status do sistema': 'Aux_Status',
        'Nota': 'NOTA',
        'Novo Descrição': 'TEXTO',
        'Novo Texto do item': 'Texto_Item',
        'Novo Texto da causa': 'Texto_Causa',
        'Modelo': 'ATIVO_TL'
    }

    # Criar uma lista de novos nomes, mantendo os originais onde não há correspondência
    colunas_convertidas = [mapa_colunas.get(
        col, col) for col in colunas_atuais]

    # Garantir que o número de nomes no mapeamento corresponda ao número de colunas
    if len(colunas_convertidas) == len(colunas_atuais):
        # Renomear as colunas no DataFrame `df_QUERY_notasExport`
        df_QUERY_notasExport_filtred.columns = colunas_convertidas
    else:
        print(
            "Erro: O número de colunas mapeadas não corresponde ao número de colunas no DataFrame.")

    df_QUERY_notasExport_filtred['BASE'] = "SAP_Arquivo_LOCAL"
    df_QUERY_z369['BASE'] = "SAP_STAGE"

    df_concatenado = pd.concat(
        [df_QUERY_z369, df_QUERY_notasExport_filtred])

    df_resultado_Export_z369_final = df_concatenado.drop_duplicates(
        subset=['NOTA'], keep='last')

else:
    user_info = "Sem dados Locais - APENAS STAGE"
    print("nota_export NOK")

if len(df_QUERY_TELA_164_FOTO) > 1000:
    print("df_QUERY_TELA_164_FOTO OK ")
else:
    print("df_QUERY_TELA_164_FOTO NOK ")

if len(df_QUERY_z369) > 1000:
    print("df_QUERY_z369 OK ")
else:
    print("df_QUERY_z369 NOK ")

# ponto chave para textes
df_QUERY_z369 = df_resultado_Export_z369_final
df_QUERY_z369_bkp = df_resultado_Export_z369_final

# ponto chave para textes

df_QUERY_TELA_164_FOTO["ATIVO_TL"] = df_QUERY_TELA_164_FOTO["VAGAO"].astype(
    str) + df_QUERY_TELA_164_FOTO["SERIE"].astype(str)
df_QUERY_TELA_164_FOTO['ATIVO_TL'] = df_QUERY_TELA_164_FOTO['VAGAO'].astype(str).str.zfill(7) + df_QUERY_TELA_164_FOTO['SERIE']
df_stage_merge = pd.merge(df_QUERY_TELA_164_FOTO,
                            df_QUERY_z369, on='ATIVO_TL', how='left') # ERRO aqui !!

# df_stage_merge_teste = df_stage_merge[(df_stage_merge['TREM'] == 'C65')]
# print("C65 - 2")
# print(len(df_stage_merge_teste))

df_stage_merge['NR_OS'] = df_stage_merge['NR_OS'].apply(
    lambda x: str(int(x)) if pd.notnull(x) else '')
df_stage_merge = df_stage_merge[(df_stage_merge['Aux_Status'] != "Em processamento")]



# Reiniciar opções de exibição do pandas
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')
pd.reset_option('display.width')
pd.reset_option('display.max_colwidth')

# Criar a pivot table, preenchendo valores ausentes com 0
Pontuacao_trem = pd.pivot_table(
    df_stage_merge, index='NR_OS', columns='TP_NOTA', aggfunc='size', fill_value=0)

# Resetar o índice para facilitar o acesso às colunas
Pontuacao_trem = Pontuacao_trem.reset_index()

# Garantir que todos os NR_OS possíveis estejam no dataframe
# Adicionar NR_OS ausentes, preenchendo com zeros
nr_os_unicos = df_stage_merge['NR_OS'].unique()
Pontuacao_trem = Pontuacao_trem.set_index('NR_OS').reindex(nr_os_unicos, fill_value=0).reset_index()

# Exibir para testes
print(Pontuacao_trem)

# Teste específico com NR_OS 1989764
# Pontuacao_trem_teste = Pontuacao_trem[(Pontuacao_trem['NR_OS'] == 1989764)]
# print("C65 - 3")
# print(len(Pontuacao_trem_teste))

# Mesclar com as informações de TREM e NR_OS
Pontuacao_trem = Pontuacao_trem.merge(
    df_stage_merge[['TREM', 'NR_OS']].drop_duplicates(), on='NR_OS', how='left'
)

# Ordenar, remover duplicatas e selecionar colunas relevantes
Pontuacao_trem = Pontuacao_trem.sort_values(
    by='M2', ascending=False).drop_duplicates().reset_index(drop=True)

Pontuacao_trem = Pontuacao_trem[['TREM', 'M1', 'M2', 'NR_OS']]

# Converter a coluna TREM para string e filtrar valores não desejados
Pontuacao_trem['TREM'] = Pontuacao_trem['TREM'].astype(str)
Pontuacao_trem = Pontuacao_trem[(Pontuacao_trem['TREM'] != "None")]

# Preparar o dataframe final para renderizar
df = Pontuacao_trem

# Mesclar dados adicionais para DF_STAGE_MERGE
df_QUERY_TELA_164_FOTO["ATIVO_TL"] = df_QUERY_TELA_164_FOTO["VAGAO"].astype(
    str) + df_QUERY_TELA_164_FOTO["SERIE"].astype(str)
df_stage_merge = pd.merge(df_QUERY_TELA_164_FOTO,
                        df_QUERY_z369, on='ATIVO_TL', how='left')
df_stage_merge['NR_OS'] = df_stage_merge['NR_OS'].apply(
    lambda x: str(int(x)) if pd.notnull(x) else ''
)
df_stage_merge['DT_NOTA'] = pd.to_datetime(
    df_stage_merge['DT_NOTA'], dayfirst=True, errors='coerce'
).dt.strftime('%Y-%m-%d')

# Apenas para teste, exportar CSV
# df_stage_merge.to_csv('files//df_stage_merge.csv', index=False)

leituradf_QUERY_z369
leituradf_notasExport
leituradf_QUERY_TELA_164_FOTO
nota_export OK 
df_QUERY_TELA_164_FOTO OK 
df_QUERY_z369 OK 


C:\Users\leona\AppData\Local\Temp\ipykernel_23176\126956414.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_QUERY_notasExport_filtred['BASE'] = "SAP_Arquivo_LOCAL"


TP_NOTA    NR_OS   M1  M2  M6
0        1989806  109   1   0
1        1990020    0   0   0
2        1989764   78  11   0
3        1990047   44   0   0
4        1989790  127   3   0
..           ...  ...  ..  ..
196      1989909    0   0   0
197      1952692   38   0   0
198      1989747    0   0   0
199      1989477    0   0   0
200      1986081    0   0   0

[201 rows x 4 columns]


In [ ]:
df_QUERY_TELA_164_FOTO['ATIVO_TL2'] = 
df_QUERY_TELA_164_FOTO

,VAGAO,ID_VAGAO,SERIE,TREM,SEQUENCIA,LOCAL,ORIGEM,DESTINO,NR_OS,DT_CARGA,COD_LINHA,ATIVO_TL,ATIVO_TL2
0,941247,324027,HFT,I40,14.0,ZPS,TRO,ZEV,1989806.0,1733121421000,None,0 941247\n1 6275451\n2 ...,0941247
1,6275451,297106,PED,G02,26.0,LFP,SCA,LDP,1990020.0,1733121421000,None,0 941247\n1 6275451\n2 ...,6275451
2,606910,351206,PCT,C65,14.0,TGR,ICB,TRO,1989764.0,1733121421000,None,0 941247\n1 6275451\n2 ...,0606910
3,3528880,376285,HTT,X32,4.0,ZPT,PSS,PSN,1990047.0,1733121421000,None,0 941247\n1 6275451\n2 ...,3528880
4,8403554,360318,HPT,I37,46.0,TBA,PSN,TRO,1989790.0,1733121421000,None,0 941247\n1 6275451\n2 ...,8403554
...,...,...,...,...,...,...,...,...,...,...,...,...,...
40224,319163,354495,HPT,I50,11.0,ZFN,TRO,ZEV,1989898.0,1733121421000,None,0 941247\n1 6275451\n2 ...,0319163
40225,569089,365883,TCT,None,120.0,PVI,---,---,NaN,1733121421000,L4,0 941247\n1 6275451\n2 ...,0569089
40226,6368891,304895,FHD,None,337.0,LST,LFC,LMG,NaN,1733121421000,L999,0 941247\n1 6275451\n2 ...,6368891
40227,338184,358965,HPT,I93,28.0,ZEC,PCZ,TRO,1989993.0,1733121421000,None,0 941247\n1 6275451\n2 ...,0338184
